# Aura API Demo

## 1. Installation of needed libraries and Seetup

#### Install libraries and restart kernel

In [ ]:
%pip install python-dotenv
%pip install --user graphdatascience
%pip install --user neo4j
%pip install --user IProgress
%pip install --user tqdm

import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

#### Import libraries

In [ ]:
from dotenv import dotenv_values
from urllib.parse import urljoin
from pprint import pprint as pp
import subprocess
import datetime
import json
from pprint import pprint as pp
from graphdatascience import GraphDataScience

#### Set filenames and defaults

In [ ]:
# Default directory to store authentication and API token files. This should be added to the .gitignore file if within a repository
secret_path = '.secret'

# This file should already exist before running this code
api_credentials_file = secret_path + '/' + '/aura_ent_api.txt'

# This file will be created upon API authentication
token_file = secret_path + "/api_bearer_token"

# This will be used as the basis of Aura API requests
api_base = 'https://api.neo4j.io/'

# This file will be created upon deployment of a new Aura instance
neo4j_connections_file = secret_path + '/' + 'aura_connection.txt'

## 2. Authenticate Aura API

#### Helper functions

In [ ]:
# This function looks for a bearer token and refreshes if it does not exist or is expired
def refresh_token(api_creds, api_base):
    api_endpoint = urljoin(api_base, '/oauth/token')

    curl_cmd = "curl --request POST '{}' --user '{}:{}' --header 'Content-Type: application/x-www-form-urlencoded' --data-urlencode 'grant_type=client_credentials'".format(api_endpoint, api_creds['CLIENT_ID'], api_creds['CLIENT_SECRET'], api_creds['CLIENT_NAME'])
    result = json.loads(subprocess.check_output(curl_cmd, shell=True))
    access_token = result['access_token']
    expires_in = result['expires_in']

    now = datetime.datetime.now()
    expiration = (now + datetime.timedelta(0, expires_in)).isoformat()

    bearer_token = {
        'access_token': access_token,
        'expiration': expiration
    }

    # Save the bearer token to a file
    token_file = '.secret/api_bearer_token'
    with open(token_file, "w") as outfile:
        json.dump(bearer_token, outfile, indent=4)

    access_token = bearer_token['access_token']
    return access_token
    

#### Authenticate API and retrieve or refresh bearer token

In [ ]:
aura_creds = dotenv_values(api_credentials_file)
for item in aura_creds:
  print(item, '=', aura_creds[item])
print()
access_token = refresh_token(aura_creds, api_base)
print('access_token:')
print(access_token)
print()

#### Test to confirm access to Aura tenant

In [ ]:
api_endpoint = urljoin(api_base, '/v1/tenants')
print('api_endpoint:', api_endpoint)

list_cmd = "curl -s -X 'GET' '{}' -H 'accept: application/json' -H 'Authorization: Bearer {}'".format(api_endpoint, access_token)
list_tenants = dict(json.loads(subprocess.check_output(list_cmd, shell=True)))['data']

aura_tenants = {}
for item in list_tenants:
    aura_tenants[item['id']] = {}
    aura_tenants[item['id']]['id'] = item['id']
    aura_tenants[item['id']]['name'] = item['name']

print()
print('Aura Tenants:')
print('-------------')
for tenant in aura_tenants:
    print(aura_tenants[tenant])
print()

## 3. Deploy a new Aura instance

#### Helper functions

In [ ]:
def tenant_info(access_token, api_base, tenant_id):
    api_endpoint = urljoin(api_base, '/v1/tenants/' + tenant_id)
    tenant_data = {}
    info_cmd = "curl -s -X 'GET' '{}' -H 'accept: application/json' -H 'Authorization: Bearer {}'".format(api_endpoint, access_token)
    tenant_info = json.loads(subprocess.check_output(info_cmd, shell=True))['data']

    for item in tenant_info:
        tenant_data[tenant_info['id']] = {}
        tenant_data[tenant_info['id']]['tenant_id'] = tenant_info['id']
        tenant_data[tenant_info['id']]['tenant_name'] = tenant_info['name']
        tenant_data[tenant_info['id']]['instance_configurations'] = list(tenant_info['instance_configurations'])
        
    return tenant_data

def deploy_instance(access_token, api_base, instance_name, tenant_id, cloud_provider, region='europe-west1', aura_type='professional-ds', neo4j_version='5', size='16GB'):
    api_endpoint = urljoin(api_base, '/v1/instances')

    instance_details = {}
    print('\nDeployment specifications:')
    print('neo4j_version:', neo4j_version)
    print('region:', region)
    print('memory:', size)
    print('name:', instance_name)
    print('type:', aura_type)
    print('tenant_id', tenant_id)
    print('cloud_provider:', cloud_provider)

    request_body = {
        "version": neo4j_version,
        "region": region,
        "memory": size,
        "name": instance_name,
        "type": aura_type,
        "tenant_id": tenant_id,
        "cloud_provider": cloud_provider
    }

    json_request_body = json.dumps(request_body)

    curl_cmd = "curl -s -X 'POST' '{}' -H 'accept: application/json' -H 'Authorization: Bearer {}' -H 'Content-Type: application/json'".format(api_endpoint, access_token)
    curl_cmd += " -d '{}'".format(json_request_body)

    api_response = json.loads(subprocess.check_output(curl_cmd, shell=True))
    print(api_response)
    response_data = api_response['data']

    instance_details['id'] = response_data['id']
    instance_details['name'] = response_data['name']
    instance_details['connection_url'] = response_data['connection_url']
    instance_details['password'] = response_data['password']
    instance_details['username'] = response_data['username']
    instance_details['cloud_provider'] = response_data['cloud_provider']
    instance_details['region'] = response_data['region']
    instance_details['tenant_id'] = response_data['tenant_id']
    instance_details['type'] = response_data['type']

    return instance_details

def instance_info(access_token, api_base, instance_id):
    api_endpoint = urljoin(api_base, '/v1/instances/' + instance_id)
    instance_data = {}

    info_cmd = "curl -s -X 'GET' '{}' -H 'accept: application/json' -H 'Authorization: Bearer {}'".format(api_endpoint, access_token)

    api_response = json.loads(subprocess.check_output(info_cmd, shell=True))

    if 'data' in api_response:
        response_data = api_response['data']
        instance_status = response_data['status']

    return response_data

#### Check tenant config

In [ ]:
tenant_id = "7b9f421f-d9bb-5639-a115-f24dcdf7d17e"
tenant_data = tenant_info(access_token, api_base, tenant_id)
tenant_configs = tenant_data[tenant_id]['instance_configurations']
print('tenant_id:', tenant_data[tenant_id]['tenant_id'])
print('tenant_name:', tenant_data[tenant_id]['tenant_name'])
print('instance_configurations:')
for config in tenant_configs:
    print('\t', config['cloud_provider'], config['memory'], 'Neo4jV' + config['version'], config['region'], config['region_name'], config['type'])

#### Configure Aura instance

In [ ]:
# Set configuration for Aura instance
aura_instance_info = {
    'tenant_id': tenant_id,
    'name': "leerazo_aura_api_demo",
    'cloud_provider': 'gcp',
    'size': '8GB',
    'region': 'europe-west1',
    'type': 'enterprise-ds',
    'neo4j_version': '5'
}

pp(aura_instance_info)

#### Deploy a new Neo4j instance according to our configuration

In [ ]:
instance_details = deploy_instance(access_token, api_base, aura_instance_info['name'], tenant_id, aura_instance_info['cloud_provider'], region=aura_instance_info['region'], aura_type=aura_instance_info['type'], neo4j_version=aura_instance_info['neo4j_version'], size=aura_instance_info['size'])

NEO4J_URI = instance_details['connection_url']
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = instance_details['password']

print('\ninstance_details:')
for item in instance_details:
    print(item, '=', instance_details[item])
    
print()
print('NEO4J_URI:', NEO4J_URI)
print('NEO4J_USERNAME:', NEO4J_USERNAME)
print('NEO4J_PASSWORD:', NEO4J_PASSWORD)

#### Save the Aura credentials to a file

In [ ]:
AURA_INSTANCEID = instance_details['id']
AURA_INSTANCENAME = instance_details['name']

# Save instance authentication details to a file
export_text = ""
export_text += "NEO4J_URI = " + NEO4J_URI + "\n"
export_text += "NEO4J_USERNAME = " + NEO4J_USERNAME + "\n"
export_text += "NEO4J_PASSWORD = " + NEO4J_PASSWORD + "\n"
export_text += "AURA_INSTANCEID = " + AURA_INSTANCEID + "\n"
export_text += "AURA_INSTANCENAME = " + AURA_INSTANCENAME + "\n"

#connection_filename = secret_path + "/" + AURA_INSTANCENAME + "-" + AURA_INSTANCEID + "-connection.txt"

connection_filename = secret_path + '/' + "aura_connection.txt"

print(export_text)
f = open(connection_filename, "w")
f.write(export_text)
f.close()

#### Confirm the instance details

In [ ]:
instance_details = instance_info(access_token, api_base, instance_details['id'])
print('instance_details:')
print()
for item in instance_details:
    print('\t' + item, '=', instance_details[item])

## 4. Load some data into Neo4j

#### Connect to the Neo4j instance and confirm everything is working

In [ ]:
print('NEO4J_URI:', NEO4J_URI) 
print('NEO4J_USERNAME:', NEO4J_USERNAME) 
print('NEO4J_PASSWORD:', NEO4J_PASSWORD) 

gds = GraphDataScience(
    NEO4J_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD),
    aura_ds=True
)
gds.set_database('neo4j')
print()
print('Checking GDS version:')
gds.run_cypher('RETURN gds.version() as version')

#### Load some data from a CSV file in a Google Storage bucket
We will use a very basic demo dataset that represents the stations and connections of the London public transport network

In [ ]:
# First we'll create some nodes to represent the stations
gds.run_cypher('''
LOAD CSV WITH HEADERS FROM 'https://storage.googleapis.com/leerazo-demos/london_transport/datasets/London_stations.csv' AS row
MERGE (s:Station {latitude:toFloat(row.Latitude), longitude:toFloat(row.Longitude), name:row.Station, zone:row.Zone})
RETURN count(s) as stations
''')

In [ ]:
# Next we'll connect the stations according to the various transport lines (i.e. "Jubilee", "Bakerloo", "Overground", etc)
gds.run_cypher('''
LOAD CSV WITH HEADERS FROM 'https://storage.googleapis.com/leerazo-demos/london_transport/datasets/London_tube_lines.csv' as row
MATCH (a:Station), (b:Station) WHERE a.name = row.From_Station AND b.name = row.To_Station
CALL apoc.create.relationship(a, toUpper(row.Tube_Line), {}, b)
YIELD rel as rel1
CALL apoc.create.relationship(b, toUpper(row.Tube_Line), {}, a)
YIELD rel as rel2
RETURN count(rel1) + count(rel2) AS relationships;
''')

In [ ]:
# Let's run a test query to list some stations connected on the Jubilee line
gds.run_cypher('''
MATCH (a)-[r:JUBILEE]-()
RETURN a.name LIMIT 25
''')

#### Clean up

In [ ]:
# This command will delete the graph
gds.run_cypher('''
MATCH (n) DETACH DELETE n
''')